# 1.2 | Data Acquisition: Manual BART Addendum
* [01 API Data Requests](01_API_pulls.ipynb)
* _[01.1 Additional BART Data](01_v2_bart.ipynb.ipynb)_
* [02 Initial EDA](02_EDA.ipynb)
* [03 First Model: PROPHET](03_prophet.ipynb)
---

### [BART](bart.gov) Reporting changed in mid-2018
* this notebook extracts monthly ridership counts for `August 2018` through `April 2022` from monthly reports archived in yearly directories, appends all to file that is appened to data obtained in previous API requests. 


In [2]:
##### BASIC IMPORTS 
import numpy as np
import pandas as pd
import glob
import os

In [9]:
path = '../data/raw/bart/'
file = 'customer-ridership.csv'

filename = path + file
df = pd.read_csv(filename)

df.shape

(259, 4)

In [10]:
df.tail()

,FiscalYear,FiscalMonth,RIDERSHIP WEEKAVG,RIDERSHIP GOAL
254,2018,3,422201.0,437728.0
255,2018,4,426492.0,439970.0
256,2018,5,423264.0,430308.0
257,2018,6,391219.0,405048.0
258,2018,7,395222.0,423540.0


In [11]:
df.rename(columns = {'Unnamed: 0': 'exit '}, inplace=True)          # rename column 0 column to dictionary 
df.drop(columns = ['RIDERSHIP GOAL'], inplace = True)

new_col = {
    'RIDERSHIP WEEKAVG' : 'ridership',
    'FiscalMonth':'month',
    'FiscalYear':'year', 
}

df.rename(columns = new_col, inplace = True)
bart = df
bart.head()

,year,month,ridership
0,1997,1,251524.0
1,1997,2,256261.0
2,1997,3,263602.0
3,1997,4,264442.0
4,1997,5,265244.0


In [12]:
# add new cols from old date colum
bart['day'] = '01'
# bart['month'] = bart['month'].apply(lambda x: '0' + str(x) if x < 10 else x )
bart['ds'] = bart['year'].astype(str) + '-' + bart['month'].astype(str) + '-01'

bart['ridership'] = 4*bart['ridership'].astype(int) # ridershiop is weekly, assume 4-week months

In [13]:
bart['date'] = pd.to_datetime(bart['ds'])
bart.index = bart['date']
bart.sort_index(inplace=True)

bart_out = bart[['ds', 'ridership']]

In [14]:
bart_out.tail()

,ds,ridership
date,,
2018-03-01,2018-3-01,1688804
2018-04-01,2018-4-01,1705968
2018-05-01,2018-5-01,1693056
2018-06-01,2018-6-01,1564876
2018-07-01,2018-7-01,1580888


### Manually extracting data for BART 2018 forward 

This function: 
* goes through one folder
* goes through each file 
* gets a monthly ridership value
* returns values for a a year

This function is called in a loop that iterates over a list of 5 years to concact all data

In [15]:
def get_monthly_bart(year):
    path = '../data/raw/bart/'
    folder = 'ridership_' + str(year) + '/'

    files = os.listdir( path + folder )

    df_year = []

    for file in files:
        filename = path + folder + file
        df_in = pd.read_excel(filename, None, skiprows = 1) 
        
        rides = df_in['Total Trips OD']['Exits'][50] 
        rides = int(rides/7)   # divide by 7 days

        ds = str(year) + '-' + file[14:16] + '-01'

        bart_month = (ds, rides)
        df_year.append(bart_month)

    return(df_year)

In [16]:
years = [2018, 2019, 2020, 2021, 2022]
all_years = []

for year in years:
    each_year = get_monthly_bart(year)
    all_years.extend(each_year)

In [ ]:
# pip install openpyxl

In [17]:
all_years = pd.DataFrame(all_years)
all_years.columns = ('ds', 'ridership')
all_years.head()

all_years['date'] = pd.to_datetime(all_years['ds'])
all_years.sort_index(inplace=True)
all_years = all_years.set_index('date')

In [18]:
merged = pd.concat([bart_out['2000-01-01':], all_years])
merged.sort_index(inplace=True)
merged.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 268 entries, 2000-01-01 to 2022-04-01
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ds         268 non-null    object
 1   ridership  268 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 6.3+ KB


In [19]:
merged.head()

,ds,ridership
date,,
2000-01-01,2000-1-01,1178324
2000-02-01,2000-2-01,1178420
2000-03-01,2000-3-01,1198004
2000-04-01,2000-4-01,1220648
2000-05-01,2000-5-01,1222712


In [20]:
merged.tail()

,ds,ridership
date,,
2021-12-01,2021-12-01,380838
2022-01-01,2022-01-01,312346
2022-02-01,2022-02-01,360330
2022-03-01,2022-03-01,477714
2022-04-01,2022-04-01,482207


In [21]:
merged.to_csv('../data/processed/bart.csv', index = False)